# ExAI - Explainable Corgy (Cardigan) seperator 🐶

We use [Contrastive GradCAM](https://xai-blog.netlify.app/docs/groups/contrastive-grad-cam-consistency/#contrastive-grad-cam-consistency-loss)
and [Layerwise Relevance Propagation](https://github.com/kaifishr/PyTorchRelevancePropagation) to explain the difference between Corgys and Cardigans.

 - We leverage [Standford ImageNet Dog Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) for fintuning [ResNet](https://pytorch.org/hub/pytorch_vision_resnet/#model-description).
 - suspects: [Pembroke](https://de.wikipedia.org/wiki/Welsh_Corgi_Pembroke) | [Cardigan](https://de.wikipedia.org/wiki/Welsh_Corgi_Cardigan)

## The Process..

 1. Data/Dependency Loading and Transformation.
 2. Model Definition.
 3. Loss Functions and Optimization definitions.
 4. Actual Training loop.
 5. Contrastive Grad-CAM Visualization.
 6. Layerwise Relevance Propagation.
 7. Save finetuned model.



## 1. Data/Dependency Loading and Transformation

In [ ]:
#@title Install dependencies.. [once]
%pip install typing matplotlib numpy pandas torch torchvision tqdm -q

In [10]:
#@title load up dependencies.. [every run]
from typing import Dict

%matplotlib inline
import matplotlib.pyplot as plt

import os
from PIL import Image

# Numpy
import numpy as np

# Pandas
import pandas as pd

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

# Progress bar
from tqdm import tqdm

# Torchvision
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as TF

# Datasets
from torchvision.datasets import Imagenette

# Datasets utils
from torchvision.datasets.utils import download_url, extract_archive

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Active device for training:", device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 837.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.8 MB/s eta 0:00:00
Active device for training: cpu


In [7]:
# If you want to persist data, you have to store it on g-drive or github.
# ..unfortunately you have to connect to g-drive manually (once every session).
#@title Setup Google Drive for persistent storage [optional | every run]

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}

if USE_GOOGLE_DRIVE:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

!echo -= Done =-


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
-= Done =-


In [2]:
#@title Download training data [once]

DOWNLOAD_DIR = "/content/drive/MyDrive/xAI-Corgis" # @param{type:"string"}

import os
import requests

def download_data(url, path):
    """
    Downloads data from a given URL and stores it in a designated path.

    Args:
        url: The URL of the data to download.
        path: The path to store the downloaded data.
    """

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(path), exist_ok=True)

    # Download the data
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Save the data to the specified path
    filename = os.path.basename(url)
    filepath = os.path.join(path, filename)

    with open(filepath, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"Downloaded {filename} to {filepath}")


os.makedirs(DOWNLOAD_DIR, exist_ok=True)

download_data("http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar", DOWNLOAD_DIR)
# download_data("http://vision.stanford.edu/aditya86/ImageNetDogs/train_data.mat", TRAINING_SETS_TARGET)
# download_data("http://vision.stanford.edu/aditya86/ImageNetDogs/test_data.mat", TRAINING_SETS_TARGET)


Downloaded images.tar to /content/drive/MyDrive/xAI-Corgis/images.tar


In [ ]:
#@title

In [8]:
#@title Extract training data to target [every training run]

DOWNLOAD_DIR = "/content/drive/MyDrive/xAI-Corgis" # @param{type:"string"}
EXTRACT_TRAINING_SETS_TO = "/content/dogs" #@param {type:"string"}

import os
import tarfile

os.makedirs(EXTRACT_TRAINING_SETS_TO, exist_ok=True)

for file in os.listdir(DOWNLOAD_DIR):
    if file.endswith(".tar"):
        file_path = os.path.join(DOWNLOAD_DIR, file)
        with tarfile.open(file_path, "r") as tar:
            tar.extractall(path=EXTRACT_TRAINING_SETS_TO)
            print(f"{file} successfully extracted to: '{EXTRACT_TRAINING_SETS_TO}'.")

images.tar successfully extracted to: '/content/dogs'.


## 2. Model Definition

In [ ]:
#@title 2.1. Prepare fine tuning of ResNet50 in PyTorch

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import scipy.io as sio
import pandas as pd
import os

# Assuming you have the .mat files downloaded as shown in the previous code.

# Define paths to your .mat files
train_data_path = os.path.join(TRAINING_SETS_TARGET, 'train_data.mat')
test_data_path = os.path.join(TRAINING_SETS_TARGET, 'test_data.mat')

# Custom Dataset class to handle .mat files
class MatDataset(Dataset):
    def __init__(self, mat_file_path, transform=None, image_field_name="images", label_fiels_name="labels"):
      """
        Please make sure your field_names in your .mat file are according to source, wont be checked!
      """
      self.data = sio.loadmat(mat_file_path) # Load .mat file using scipy.io
      self.images = self.data[image_field_name] # Adapt based on actual .mat structure
      self.labels = self.data[label_fiels_name] # "
      self.transform = transform

    def __len__(self):
      return len(self.images)

    def __getitem__(self, idx):
      image = self.images[idx]
      label = self.labels[idx]

      if self.transform:
          image = self.transform(image)
      return image, label

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = MatDataset(train_data_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = MatDataset(test_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for your specific number of classes
num_classes = len(set(train_dataset.labels)) # Replace with your number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Define loss function, optimizer, and device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using following device: {device}")

# load..
model = model.to(device)

In [ ]:
#@title 2. Run Fine-tuning

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fine-tuning loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Evaluation (example)
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test data: {100 * correct / total:.2f}%")

In [ ]:
#@title Save the finetuned model
torch.save(model.state_dict(), 'resnet50_finetuned.pth')

## 3. Loss Functions and Optimization definitions.


## 4. Actual Training loop

## 5. Contrastive Grad-CAM Visualization

Explanation by transforming same images and evaluating the result.  
TODO: Preferably by applying a before/after view.

## 6. Layerwise Relevance Propagation

## 7. Save finteuned model